In [ ]:
import datetime ,os

In [1]:
code = 'B120'
pickle_path = 'C:/PICKLE/'
parameter_path = f'Parameter_{code}.csv'
meta_data_path = f"Parameter_{code}_MetaData.csv"
output_csv_path = f'{code}_output/'

from pgcbacktest.BtParameters import *
from pgcbacktest.BacktestOptions import *

try:
    parameter, parameter_len = get_parameter_data(code, parameter_path)
    meta_data, meta_row_nos = get_meta_data(code, meta_data_path)
    os.makedirs(output_csv_path, exist_ok=True)
except Exception as e:
    input(str(e))

In [2]:
def b120(bt, start_time, end_time, orderside, method, sl, ut_sl, om):
    try:
        start_dt = datetime.datetime.combine(bt.current_date, start_time)
        end_dt = datetime.datetime.combine(bt.current_date, end_time)
        end_dt_1m = end_dt + datetime.timedelta(minutes=10)

        ce_scrip, pe_scrip, ce_price, pe_price, future_price, start_dt = bt.get_strike(start_dt, end_dt, om=om)
        if ce_scrip is None: return None
        
        from_candle_close = True if method == 'CC' else False

        entry_time = start_dt
        ce_open, ce_high, ce_low, ce_close, ce_sl_price, ce_sl_flag, _, _, ce_sl_time, ce_pnl = bt.sl_check_single_leg(start_dt, end_dt, ce_scrip, sl=sl, with_ohlc=True, orderside=orderside, from_candle_close=from_candle_close)
        pe_open, pe_high, pe_low, pe_close, pe_sl_price, pe_sl_flag, _, _, pe_sl_time, pe_pnl = bt.sl_check_single_leg(start_dt, end_dt, pe_scrip, sl=sl, with_ohlc=True, orderside=orderside, from_candle_close=from_candle_close)
        ce_sl_time = ce_sl_time if ce_sl_time else end_dt_1m
        pe_sl_time = pe_sl_time if pe_sl_time else end_dt_1m

        ut_scrip, ut_open, ut_high, ut_low, ut_close, ut_sl_price, ut_sl_flag, ut_sl_time, ut_pnl = '', '', '', '', '', '', False, '', 0
        ut_sl = ut_sl if str(ut_sl) == 'TTC' else float(ut_sl)
        B_PL, TT_PL_at_SL, UT_PL_at_SL = 0, 0, 0
        
        if ce_sl_time < pe_sl_time:
            
            ut_sl_price = pe_price if str(ut_sl) == 'TTC' else None
            ut_open, ut_high, ut_low, ut_close, ut_sl_price, ut_sl_flag, _, _, ut_sl_time, ut_pnl = bt.sl_check_single_leg(ce_sl_time, end_dt, pe_scrip, sl=ut_sl, sl_price=ut_sl_price, with_ohlc=True, pl_with_slipage=False, orderside=orderside, from_candle_close=from_candle_close)

            if ut_open:
                ut_scrip = pe_scrip
                TT_PL_at_SL = ce_pnl
                UT_PL_at_SL = pe_price - ut_open - bt.Cal_slipage(pe_price)
                
                if (str(ut_sl) == 'TTC') and (ut_open > ut_sl_price):
                    ut_sl_price = pe_sl_price
                    ut_open, ut_high, ut_low, ut_close, ut_sl_price, ut_sl_flag, _, _, ut_sl_time, ut_pnl = bt.sl_check_single_leg(ce_sl_time, end_dt, pe_scrip, sl=ut_sl, sl_price=ut_sl_price, with_ohlc=True, pl_with_slipage=False, orderside=orderside, from_candle_close=from_candle_close)
            else:
                B_PL = ce_pnl + pe_pnl

        elif pe_sl_time < ce_sl_time:
            
            ut_sl_price = ce_price if str(ut_sl) == 'TTC' else None
            ut_open, ut_high, ut_low, ut_close, ut_sl_price, ut_sl_flag, _, _, ut_sl_time, ut_pnl = bt.sl_check_single_leg(pe_sl_time, end_dt, ce_scrip, sl=ut_sl, sl_price=ut_sl_price, with_ohlc=True, pl_with_slipage=False, orderside=orderside, from_candle_close=from_candle_close)

            if ut_open:
                ut_scrip = ce_scrip
                TT_PL_at_SL = pe_pnl
                UT_PL_at_SL = ce_price - ut_open - bt.Cal_slipage(ce_price)
                
                if (str(ut_sl) == 'TTC') and (ut_open > ut_sl_price):
                    ut_sl_price = ce_sl_price
                    ut_open, ut_high, ut_low, ut_close, ut_sl_price, ut_sl_flag, _, _, ut_sl_time, ut_pnl = bt.sl_check_single_leg(pe_sl_time, end_dt, ce_scrip, sl=ut_sl, sl_price=ut_sl_price, with_ohlc=True, pl_with_slipage=False, orderside=orderside, from_candle_close=from_candle_close)
            else:
                B_PL = ce_pnl + pe_pnl

        else:
            B_PL = ce_pnl + pe_pnl

        ce_sl_time = '' if ce_sl_time == end_dt_1m else ce_sl_time
        pe_sl_time = '' if pe_sl_time == end_dt_1m else pe_sl_time
        ut_sl_time = '' if ut_sl_time == end_dt_1m else ut_sl_time

        total_pnl = B_PL + TT_PL_at_SL + UT_PL_at_SL + ut_pnl
        return [code, bt.index, start_time, end_time, orderside, method, sl, cv(ut_sl), om, bt.current_date.date(), bt.current_date.day_name(), bt.dte, entry_time.time(), future_price, ce_scrip, ce_open, ce_high, ce_low, ce_close, ce_sl_flag, ce_sl_time, ce_pnl, pe_scrip, pe_open, pe_high, pe_low, pe_close, pe_sl_flag, pe_sl_time, pe_pnl, ut_scrip, ut_open, ut_high, ut_low, ut_close, ut_sl_flag, ut_sl_time, B_PL, TT_PL_at_SL, UT_PL_at_SL, ut_pnl, total_pnl]
    except Exception as e:
        print(e, [bt.index, bt.current_date, start_time, end_time, orderside, method, sl, ut_sl, om])
        return

In [3]:
for row_idx in range(len(meta_data)):

    if row_idx in meta_row_nos and meta_data.loc[row_idx, 'run']:
        try:
            meta_row = meta_data.iloc[row_idx]
            index, dte, from_date, to_date, start_time, end_time, date_lists = get_meta_row_data(meta_row, pickle_path)

            log_cols = ('P_Strategy/P_Index/P_StartTime/P_EndTime/P_OrderSide/P_Method/P_SL/P_UTSL/P_OM/Date/Day/DTE/EntryTime/Future/CE.Strike/CE.Open/CE.High/CE.Low/CE.Close/CE.SL.Flag/CE.SL.Time/CE.PNL/PE.Strike/PE.Open/PE.High/PE.Low/PE.Close/PE.SL.Flag/PE.SL.Time/PE.PNL/UT.Strike/UT.Open/UT.High/UT.Low/UT.Close/UT.SL.Flag/UT.SL.Time/BPL/TT.PL.AT.SL/UT.PL.AT.SL/UT.PNL/Total.PNL').split('/')

            for current_date in date_lists:

                file_name = f"{index} {current_date.date()} {code}"
                if not is_file_exists(output_csv_path, file_name, parameter_len):

                    t1 = datetime.datetime.now()
                    print(f"Row-{row_idx} | File-{file_name} | Total-{parameter_len}")

                    bt = IntradayBacktest(pickle_path, index, current_date, dte, start_time, end_time)
                    
                    for idx, i in enumerate(range(0, parameter_len, chunk_size), start=1):
                        chunck_file_name = f"{output_csv_path}{file_name} No-{idx}.parquet"
                        print(chunck_file_name)

                        chunk_parameter = parameter.iloc[i:i+chunk_size]
                        chunk = [b120(bt, row.entry_time, row.exit_time, row.orderside, row.method, row.sl, row.ut_sl, row.om) for row in tqdm(chunk_parameter.itertuples(), total=len(chunk_parameter), colour='GREEN')]
                        save_chunk_data(chunk, log_cols, chunck_file_name)
                        
                        del chunk
                        del chunk_parameter
                        gc.collect()

                    del bt
                    gc.collect()
                    
                    t2 = datetime.datetime.now()
                    print(t2-t1)
        except Exception as e:
            input(str(e))

Row-0 | File-NIFTY 2022-01-06 B120 | Total-4350
B120_output/NIFTY 2022-01-06 B120 No-1.parquet


100%|██████████| 4350/4350 [00:03<00:00, 1345.30it/s]


Deleting instance ... 2022-01-06 00:00:00
0:00:03.537382
Row-0 | File-NIFTY 2022-01-13 B120 | Total-4350
B120_output/NIFTY 2022-01-13 B120 No-1.parquet


 77%|███████▋  | 3328/4350 [00:01<00:00, 1742.59it/s]


KeyboardInterrupt: 

In [12]:
for row in tqdm(chunk_parameter.itertuples()):
    break

0it [00:00, ?it/s]
